# Creating the Complete Tool from Downloading the Dataset to the Final Prediction with Graphs

### Prereqs for the Tool to run

In [ ]:
# path to reference virus
zika_virus='/mnt/metagenome/Zika_brazil.fasta'

# path to a file with a list of accessions, one accession per line
acc_list='/mnt/metagenome/acc.txt'

In [ ]:
mkdir fastq

while read p; do
echo "$p" 

prefetch $p --max-size u -O /mnt/metagenome
fasterq-dump -p /mnt/metagenome/$p -e 4 -O /mnt/metagenome/fastq
# pigz -0 -p4 $p*

rm -r "$p"

done < $acc_list

# Trimming

In [ ]:
mkdir trimmed
while read p; do
echo "$p" 

if [ -f fastq/$p"_1.fastq" ] && [ -f fastq/$p"_2.fastq" ]; then
    trim_galore --cores 8 --paired -o trimmed/ fastq/$p"_1.fastq" fastq/$p"_2.fastq"
else
    trim_galore --cores 8 -o trimmed/ fastq/$p".fastq"
fi

done < $acc_list

# Expression

In [ ]:
mkdir virus_ref

In [ ]:
bowtie2-build -f $zika_virus virus_ref/zika_virus

In [ ]:
Virus=zika_virus

while read p; do
echo $p

if [ -f trimmed/$p"_1_val_1.fq" ] && [ -f trimmed/$p"_2_val_2.fq" ]; then
    time bowtie2 --local --very-sensitive-local --threads 8 -k 10 -x virus_ref/$Virus -1 trimmed/$p"_1_val_1.fq" -2 trimmed/$p"_2_val_2.fq" -S  $Virus"_"$p
else
    time bowtie2 --local --very-sensitive-local --threads 8 -k 10 -x virus_ref/$Virus -U trimmed/$p"_trimmed.fq" -S  $Virus"_"$p
fi

samtools view -S -b -@ 8 -F 260 $Virus"_"$p > $Virus"_"$p".bam"

rm $Virus"_"$p

samtools sort -@ 8 $Virus"_"$p".bam" > $Virus"_"$p"_sorted.bam"

rm $Virus"_"$p".bam"

samtools index $Virus"_"$p"_sorted.bam"

done < $acc_list

# Subgenomic Plotting

In [35]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.gridspec as gridspec

In [36]:
# Function to read numbers from a file and return them as a list
def read_acc_from_file(file_path):
    acc = []
    with open(file_path, 'r') as file:
        for line in file:
            # Strip the newline character and convert the line to an integer
            accession = str(line.strip())
            acc.append(accession)
    return acc

# Example usage
file_path = '/mnt/metagenome/acc.txt'  # Replace with the path to your file
acc_list = read_acc_from_file(file_path)

print(acc_list)

['SRR14849118', 'SRR14849121', 'SRR14849125', 'SRR14849126', 'SRR14849130', 'SRR14849131', 'SRR14849132', 'SRR14849145', 'SRR14849149']


In [37]:
%cd /mnt/metagenome

/mnt/metagenome


In [11]:
!mkdir plotting
!mkdir plotting/original_depth

mkdir: cannot create directory ‘plotting’: File exists
mkdir: cannot create directory ‘plotting/original_depth’: File exists


In [ ]:
while read p; do
echo $p

samtools depth -aa *$p"_sorted.bam" > plotting/original_depth/$p".tsv"

done < acc.txt

## Antisense sense read seperation

In [12]:
!mkdir bam_read_seperation/

In [ ]:
single() {

samtools view -@ 8 -b -F 0x10 bam_read_seperation/$p"_filtered.bam" > bam_read_seperation/$p"_sense.bam"
samtools view -@ 8 -b -f 0x10 bam_read_seperation/$p"_filtered.bam" > bam_read_seperation/$p"_antisense.bam"

samtools sort -@ 8 bam_read_seperation/$p"_sense.bam" > bam_read_seperation/$p"_sense_sorted.bam"
samtools sort -@ 8 bam_read_seperation/$p"_antisense.bam" > bam_read_seperation/$p"_antisense_sorted.bam"

rm bam_read_seperation/$p"_filtered.bam"
rm bam_read_seperation/$p"_sense.bam"
rm bam_read_seperation/$p"_antisense.bam"
}

In [ ]:
while read p; do

samtools view -@ 8 -b -q 10 *$p"_sorted.bam" > bam_read_seperation/$p"_filtered.bam"

single

done < acc.txt

## Positive and Negative Read Coverage Plots

In [14]:
!mkdir plotting/positive_depth
!mkdir plotting/negative_depth

In [ ]:
while read p; do
echo $p
samtools depth -aa bam_read_seperation/$p"_sense_sorted.bam" > plotting/positive_depth/$p".tsv"
samtools depth -aa bam_read_seperation/$p"_antisense_sorted.bam" > plotting/negative_depth/$p".tsv"
done < acc.txt

## Positive and Negative 5' start points and 3' End points

In [15]:
!mkdir read_start_end

In [ ]:
while read p; do
echo $p
samtools view bam_read_seperation/$p"_sense_sorted.bam" | cut -f 4,6 > read_start_end/$p"_sense.tsv"
samtools view bam_read_seperation/$p"_antisense_sorted.bam" | cut -f 4,6 > read_start_end/$p"_antisense.tsv"

done < acc.txt

In [38]:
%cd /mnt/metagenome/read_start_end/

/mnt/metagenome/read_start_end


In [39]:
from cigar import Cigar

# Example function to apply
def cigar_len(x):
    return Cigar(x).reference_length()

In [40]:
# Function to read numbers from a file and return them as a list
def read_acc_from_file(file_path):
    acc = []
    with open(file_path, 'r') as file:
        for line in file:
            # Strip the newline character and convert the line to an integer
            accession = str(line.strip())
            acc.append(accession)
    return acc

# Example usage
file_path = '/mnt/metagenome/acc.txt'  # Replace with the path to your file
acc_list = read_acc_from_file(file_path)

print(acc_list)

['SRR14849118', 'SRR14849121', 'SRR14849125', 'SRR14849126', 'SRR14849130', 'SRR14849131', 'SRR14849132', 'SRR14849145', 'SRR14849149']


In [41]:
for i in acc_list:
    
    positive_start = pd.read_csv(i + "_sense.tsv", sep="\t", names=["Start", "CIGAR"])
    
    positive_start['Length'] = positive_start['CIGAR'].apply(cigar_len)
    positive_start['End'] = positive_start['Start'] + positive_start['Length']
    
    positive_start = positive_start.drop('Length', axis=1)
    positive_start = positive_start.drop('CIGAR', axis=1)
    
    positive_start.to_csv(i + "_sense_edited.tsv", sep="\t", header=True, index=False)
    
    
    negative_start = pd.read_csv(i + "_antisense.tsv", sep="\t", names=["Start", "CIGAR"])
    
    negative_start['Length'] = negative_start['CIGAR'].apply(cigar_len)
    negative_start['End'] = negative_start['Start'] + negative_start['Length']
    
    negative_start = negative_start.drop('Length', axis=1)
    negative_start = negative_start.drop('CIGAR', axis=1)
    
    negative_start.to_csv(i + "_antisense_edited.tsv", sep="\t", header=True, index=False)

In [42]:
!mkdir /mnt/metagenome/plotting/negative_start
!mkdir /mnt/metagenome/plotting/positive_start

mkdir: cannot create directory ‘/mnt/metagenome/plotting/negative_start’: File exists
mkdir: cannot create directory ‘/mnt/metagenome/plotting/positive_start’: File exists


In [43]:
for i in acc_list:
    original_depth = pd.read_csv("/mnt/metagenome/plotting/original_depth/" + i + ".tsv", sep="\t", names=["Virus", "Position", "Count"])
    full_range = pd.DataFrame({'Start': range(original_depth['Position'].min(), original_depth['Position'].max() + 1)})
    
    positive_start = pd.read_csv(i + "_sense_edited.tsv", sep="\t")
    negative_start = pd.read_csv(i + "_antisense_edited.tsv", sep="\t")
    
    counts_pos = positive_start['Start'].value_counts().reset_index()
    counts_neg = negative_start['Start'].value_counts().reset_index()
    
    # Rename the columns for clarity
    counts_pos.columns = ['Start', 'Count']
    counts_neg.columns = ['Start', 'Count']
    
    counts_pos = counts_pos.sort_values(by=['Start'])
    counts_neg = counts_neg.sort_values(by=['Start'])

    counts_pos_complete = pd.merge(full_range, counts_pos, on='Start', how='left').fillna(0)
    counts_neg_complete = pd.merge(full_range, counts_neg, on='Start', how='left').fillna(0)
    
    counts_neg_complete.to_csv("/mnt/metagenome/plotting/negative_start/" + i + ".tsv", sep="\t", header=False, index=False)
    counts_pos_complete.to_csv("/mnt/metagenome/plotting/positive_start/" + i + ".tsv", sep="\t", header=False, index=False)

In [24]:
!mkdir /mnt/metagenome/plotting/negative_end
!mkdir /mnt/metagenome/plotting/positive_end

In [44]:
for i in acc_list:
    original_depth = pd.read_csv("/mnt/metagenome/plotting/original_depth/" + i + ".tsv", sep="\t", names=["Virus", "Position", "Count"])
    full_range = pd.DataFrame({'End': range(original_depth['Position'].min(), original_depth['Position'].max() + 1)})
    
    positive_end = pd.read_csv(i + "_sense_edited.tsv", sep="\t")
    negative_end = pd.read_csv(i + "_antisense_edited.tsv", sep="\t")
    
    counts_pos = positive_end['End'].value_counts().reset_index()
    counts_neg = negative_end['End'].value_counts().reset_index()
    
    # Rename the columns for clarity
    counts_pos.columns = ['End', 'Count']
    counts_neg.columns = ['End', 'Count']
    
    counts_pos = counts_pos.sort_values(by=['End'])
    counts_neg = counts_neg.sort_values(by=['End'])

    counts_pos_complete = pd.merge(full_range, counts_pos, on='End', how='left').fillna(0)
    counts_neg_complete = pd.merge(full_range, counts_neg, on='End', how='left').fillna(0)
    
    counts_neg_complete.to_csv("/mnt/metagenome/plotting/negative_end/" + i + ".tsv", sep="\t", header=False, index=False)
    counts_pos_complete.to_csv("/mnt/metagenome/plotting/positive_end/" + i + ".tsv", sep="\t", header=False, index=False)

# Plotting

In [45]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.gridspec as gridspec
import ruptures as rpt
import pickle

In [46]:
# Function to read numbers from a file and return them as a list
def read_acc_from_file(file_path):
    acc = []
    with open(file_path, 'r') as file:
        for line in file:
            # Strip the newline character and convert the line to an integer
            accession = str(line.strip())
            acc.append(accession)
    return acc

# Example usage
file_path = '/mnt/metagenome/acc.txt'  # Replace with the path to your file
acc_list = read_acc_from_file(file_path)
print(acc_list)

['SRR14849118', 'SRR14849121', 'SRR14849125', 'SRR14849126', 'SRR14849130', 'SRR14849131', 'SRR14849132', 'SRR14849145', 'SRR14849149']


In [47]:
%cd /mnt/metagenome/plotting/

/mnt/metagenome/plotting


In [30]:
reference_virus = {
"ERR1802071": "Zika_brazil","ERR1802073": "Zika_brazil","ERR1802074": "Zika_brazil","ERR1802079": "Zika_brazil","SRR12615449": "Zika_brazil","SRR12615452": "Zika_brazil","SRR12615453": "Zika_brazil","SRR12615466": "Zika_brazil","SRR12615469": "Zika_brazil","SRR12615470": "Zika_brazil","SRR13084686": "Zika_brazil","SRR13084689": "Zika_brazil","SRR13084692": "Zika_brazil","SRR13084695": "Zika_brazil","SRR21109253": "Zika_brazil","SRR21109254": "Zika_brazil","SRR21109257": "Zika_brazil","SRR7660688": "Zika_brazil","SRR7660689": "Zika_brazil","SRR7660692": "Zika_brazil","SRR7660693": "Zika_brazil","SRR7660696": "Zika_brazil","SRR7660698": "Zika_brazil","SRR7660703": "Zika_brazil","SRR7660704": "Zika_brazil","SRR7660705": "Zika_brazil","SRR7660718": "Zika_brazil","SRR7660719": "Zika_brazil","SRR7660720": "Zika_brazil","SRR7660721": "Zika_brazil","SRR7660722": "Zika_brazil","SRR7660723": "Zika_brazil","SRR7660724": "Zika_brazil","SRR7660725": "Zika_brazil","SRR7660726": "Zika_brazil","SRR7660739": "Zika_brazil","SRR7660740": "Zika_brazil","SRR7660741": "Zika_brazil","SRR7660742": "Zika_brazil","SRR7660743": "Zika_brazil","SRR7660744": "Zika_brazil","SRR7660745": "Zika_brazil","SRR7660746": "Zika_brazil","SRR7660747": "Zika_brazil","SRR8155998": "Zika_brazil","SRR8156000": "Zika_brazil","SRR9106108": "Zika_brazil","SRR9106110": "Zika_brazil","SRR9106112": "Zika_brazil","SRR9106114": "Zika_brazil","SRR9106116": "Zika_brazil","SRR9106118": "Zika_brazil","SRR9106120": "Zika_brazil","SRR9106122": "Zika_brazil","SRR9106124": "Zika_brazil","SRR9106126": "Zika_brazil","SRR9106128": "Zika_brazil","SRR9106130": "Zika_brazil","SRR9106132": "Zika_brazil","SRR9106134": "Zika_brazil","SRR9106136": "Zika_brazil","SRR9106138": "Zika_brazil","SRR9106140": "Zika_brazil","SRR9106142": "Zika_brazil","SRR9106144": "Zika_brazil","SRR9106146": "Zika_brazil","SRR9610797": "Zika_brazil","SRR9610798": "Zika_brazil","SRR9610799": "Zika_brazil","SRR9610800": "Zika_brazil","SRR9610801": "Zika_brazil","SRR9610805": "Zika_brazil","SRR9971529": "Zika_brazil","SRR9971530": "Zika_brazil","SRR9971531": "Zika_brazil","SRR9971532": "Zika_brazil","SRR9971533": "Zika_brazil","SRR9971534": "Zika_brazil","SRR9971535": "Zika_brazil","SRR9971536": "Zika_brazil","SRR9971537": "Zika_brazil","SRR9971538": "Zika_brazil","SRR9971539": "Zika_brazil","SRR9971540": "Zika_brazil","SRR15923994": "Zika_uganda","SRR15923995": "Zika_uganda","SRR15923996": "Zika_uganda","SRR8155999": "Zika_uganda"
}

In [48]:
# chopped = 100
# buffer = 105
# detection_zone = 150
def classifier_model(coverage, top3peaks, chopped, buffer, detection_zone):

    mean_diff = coverage.copy()   
    
    coverage = coverage[chopped:-chopped]    

    for i in range(chopped+buffer):
        mean_diff[i][0]=0
    for i in range(len(mean_diff)-(chopped+buffer),len(mean_diff)):
        mean_diff[i][0]=0
    # for i in mean_diff:
    #     print(i)
    
    for index in range(buffer,len(coverage)-buffer):
        left_mean = coverage[:index].mean()
        right_mean = coverage[index:].mean()
        diff = abs(right_mean - left_mean)
        # print(diff)
        mean_difference = diff
        mean_diff[index+chopped][0]=mean_difference
    
    result = mean_diff.argmax()
    # plt.plot(mean_diff)
    # plt.show()
        
    if (top3peaks[0][0] - detection_zone) <= result <= (top3peaks[0][0] + detection_zone) or \
        (top3peaks[1][0] - detection_zone) <= result <= (top3peaks[1][0] + detection_zone) or \
        (top3peaks[2][0] - detection_zone) <= result <= (top3peaks[2][0] + detection_zone):
        verdict = True
    else:
        verdict = False

    return result, verdict

In [49]:
chopped = 100
buffer = 225
detection_zone = 100

for acc in acc_list:
    
    original_depth = pd.read_csv("original_depth/" + acc + ".tsv", sep="\t", names=["Virus", "Position", "Count"])
    positive_depth = pd.read_csv("positive_depth/" + acc + ".tsv", sep="\t", names=["Virus", "Position", "Count"])
    negative_depth = pd.read_csv("negative_depth/" + acc + ".tsv", sep="\t", names=["Virus", "Position", "Count"])
    positive_start = pd.read_csv("positive_start/" + acc + ".tsv", sep="\t", names=["Position", "Count"])
    negative_start = pd.read_csv("negative_start/" + acc + ".tsv", sep="\t", names=["Position", "Count"])
    positive_end = pd.read_csv("positive_end/" + acc + ".tsv", sep="\t", names=["Position", "Count"])
    negative_end = pd.read_csv("negative_end/" + acc + ".tsv", sep="\t", names=["Position", "Count"])

    #5' start peak detection
    positive_start2 = pd.read_csv("positive_start/" + acc + ".tsv", sep="\t", names=["Position", "Count"])
    positive_start2 = positive_start2.sort_values(by=['Count'],ascending=False)
    top3peaks = positive_start2.head(3)
    top3peaks = top3peaks.to_numpy()


    #Read coverage Change Point Detection
    depth = pd.read_csv("positive_depth/" + acc + ".tsv", sep="\t", names=["Virus", "Position", "Count"])
    depth = depth.drop(columns=["Virus"])
    depth.set_index("Position", inplace=True)
    
    coverage = depth['Count'].values.reshape(-1,1)

    result, verdict = classifier_model(coverage, top3peaks, chopped, buffer, detection_zone)
    
    # Create a list of the dataframes for easy iteration
    dataframes = [original_depth, positive_depth, negative_depth, positive_start, negative_start, positive_end, negative_end]
    titles = ["Zika Virus Coverage", acc + " Sense Reads Coverage",acc + " Antisense Reads Coverage",
              acc + " Sense Read 5\' End",acc + " Antisense Read 3\' End", acc + " Sense Read 3\' End", acc + " Antisense Read 5\' End"]
    colours = ["magma",None, "autumn",None,"autumn",None, "autumn"]
    
    
    
    # Create subplots: 4 rows and 2 columns, but only using the first slot for a single graph
    fig = plt.figure()
    axs = []
    gs = gridspec.GridSpec(4, 2)
    axs.append(plt.subplot(gs[0, 0]))
    axs.append(plt.subplot(gs[1, 0]))
    axs.append(plt.subplot(gs[1, 1]))
    axs.append(plt.subplot(gs[2, 0]))
    axs.append(plt.subplot(gs[2, 1]))
    axs.append(plt.subplot(gs[3, 0]))
    axs.append(plt.subplot(gs[3, 1]))
    axs.append(plt.subplot(gs[0, 1]))
    
    # Plot each DataFrame on its corresponding subplot
    for i, df in enumerate(dataframes):
        ax = axs[i]
        if i==3:
            ax.plot(top3peaks[0][0],top3peaks[0][1], "ob")
            ax.plot(top3peaks[1][0],top3peaks[1][1], "ob")
            ax.plot(top3peaks[2][0],top3peaks[2][1], "ob")
            ax.legend(['Peaks'])
        if i==1:
            ax.vlines(result,ymin=0,ymax=df['Count'].max(), color='black', linestyles='dashed', linewidth = 4)

        if i==0 or i==1 or i==2:
            # predicted 5′ end of RNA “stem-loop 2” (SL2)
            ax.vlines(10478,ymin=0,ymax=df['Count'].max(), color='blue', linestyles='dashed', linewidth = 1)

            # predicted 5′ end of RNA “stem-loop 1” (SL1)
            ax.vlines(10394,ymin=0,ymax=df['Count'].max(), color='blue', linestyles='dashed', linewidth = 1)
            

            #position from Andrew's paper: https://www.biorxiv.org/content/10.1101/112904v1.full
            
        df.plot("Position", "Count", ax=ax, colormap=colours[i], figsize=(18, 18))
        ax.set_title(titles[i])

    
    axs[7].annotate("Prediction: " + str(verdict), (0.26,0.6), fontsize=35, bbox=dict(boxstyle="round", fc="0.8"))
    axs[7].annotate("sfRNA predicted start: " + str(result), (0.16,0.35), fontsize=30, bbox=dict(boxstyle="round", fc="0.8"))
    
    # Adjust layout for better spacing
    plt.tight_layout()
    
    # plt.show()
    fig.savefig('/mnt/metagenome/zika_final_plots/' + acc + '.png')
    plt.close(fig)

### Classifier_100_100_225

precision: 0.756097561

recall: 1

f1_score: 0.861111111